In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import numpy as np
import os

In [2]:
from IPython.display import Image # For displaying images in colab jupyter cell
from PIL import Image as PilImage

## Prepare Data

In [3]:
# Load images

# training features/targets where each feature is a greyscale image with shape (512, 512)
classes = ["pituitary", "notumor", "meningioma", "glioma"]
train_folder = "../data/Training"
test_folder = "../data/Testing"

img_width = 128
img_height = 128
# load image from a folder
def load_image(folder_path):
    features = []
    targets = []
    for index, class_label in enumerate(classes):
        class_folder = os.path.join(folder_path, class_label)
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            #print(img_path)
            try:
                img = PilImage.open(img_path).convert("L")
                img = img.resize((img_height,img_width))
                img_array = np.array(img)
                features.append(img_array)
                targets.append(index)
            except Exception as e:
                print("Error loading image ", e)
    return np.array(features), np.array(targets)

# load images
train_features, train_targets = load_image(train_folder)
test_features, test_targets = load_image(test_folder)

# shapes of training/testing datasets
print("Training Features Shape: ", train_features.shape)
print("Training Targets Shape: ", train_targets.shape)
print("Testing Features Shape: ", test_features.shape)
print("Testing Targets Shape: ", test_targets.shape)

Training Features Shape:  (5712, 128, 128)
Training Targets Shape:  (5712,)
Testing Features Shape:  (1311, 128, 128)
Testing Targets Shape:  (1311,)


In [4]:
# Visualizing the first three training features (samples)

plt.figure(figsize = (10, 10))

plt.subplot(1,3,1)
plt.imshow(train_features[0], cmap = 'Greys')

plt.subplot(1,3,2)
plt.imshow(train_features[1], cmap = 'Greys')

plt.subplot(1,3,3)
plt.imshow(train_features[2], cmap = 'Greys')

In [5]:
# perform standard scaling
from sklearn.preprocessing import StandardScaler

# First flatten each image into 512*512 to convert features from 3D -> 2D arrays
train_features_flat = train_features.reshape((5712, img_width*img_height))
test_features_flat = test_features.reshape((1311, img_width*img_height))

# Use standard scaler to scale the flattened images
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features_flat).reshape((5712, img_width*img_height))
test_features = scaler.fit_transform(test_features_flat).reshape((1311, img_width*img_height))


In [6]:
# Shuffle training set
import random
features_and_targets = list(zip(train_features, train_targets))
random.shuffle(features_and_targets)
train_features, train_targets = zip(*features_and_targets)
train_features = np.array(list(train_features))
train_targets = np.array(list(train_targets))
print("Training Features Shape: ", train_features.shape)
print("Training Targets Shape: ", train_targets.shape)

# Take the first num_validation training features and targets as validation set
num_validation = 500
num_training = 5712 - 500
num_testing = 1311
validation_features = train_features[:num_validation]
validation_targets = train_targets[:num_validation]

# Take the remaining training features and targets as training set
train_features = train_features[num_validation:]
train_targets = train_targets[num_validation:]

Training Features Shape:  (5712, 16384)
Training Targets Shape:  (5712,)


In [7]:
# Reshape train/validation/test sets to conform to PyTorch's (N, Channels, Height, Width) standard for CNNs
train_features = np.reshape(train_features, (num_training, 1, img_width, img_height))
validation_features = np.reshape(validation_features, (num_validation, 1, img_width, img_height))
test_features = np.reshape(test_features, (num_testing, 1, img_width, img_height))

## Define Model

In [8]:
class CNNModel(torch.nn.Module):

    def __init__(self):

        super(CNNModel, self).__init__()

        # Second convolution layer with 16 out channels, 1 padding
        self.cnn1 = torch.nn.Conv2d(in_channels=1, out_channels=16,
                              kernel_size=3, stride=1, padding=1)

        # normalization for statbility
        self.batchnorm1 = torch.nn.BatchNorm2d(16)

        # size 2 kernel for maxpool
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # Second convolution layer with 32 out channels, 1 padding
        self.cnn2 = torch.nn.Conv2d(in_channels=16, out_channels=32,
                              kernel_size=3, stride=1, padding=1)

        # normalization for statbility
        self.batchnorm2 = torch.nn.BatchNorm2d(32)

        # size 2 kernel for maxpool
        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=2)

        # Third convolution layer with 64 out channels, 1 padding
        self.cnn3 = torch.nn.Conv2d(in_channels=32, out_channels=64,
                              kernel_size=3, stride=1, padding=1)

        # normalization for statbility
        self.batchnorm3 = torch.nn.BatchNorm2d(64)

        # size 2 kernel for maxpool
        self.maxpool3 = torch.nn.MaxPool2d(kernel_size=2)

        # Fully connected layer that takes the flattened output
        self.fc2 = torch.nn.Linear(64 * 16 * 16, 4)

    def forward(self, x):

        # YOUR CODE HERE

        # input image -> conv1 -> relu -> batchnorm -> maxpool1
        conv1_out = torch.nn.functional.relu(self.cnn1(x))
        pool1_out = self.maxpool1(self.batchnorm1(conv1_out))

        # maxpool1 output -> conv2 -> relu -> batchnorm -> maxpool2
        conv2_out = torch.nn.functional.relu(self.cnn2(pool1_out))
        pool2_out = self.maxpool2(self.batchnorm2(conv2_out))

        # maxpool2 output -> conv3 -> relu -> batchnorm -> maxpool3
        conv3_out = torch.nn.functional.relu(self.cnn3(pool2_out))
        pool3_out = self.maxpool3(self.batchnorm3(conv3_out))

        # flatten the maxpool3 output to be used as input into FCN layer
        fcn_input = pool3_out.view(pool3_out.size(0), -1)

        # Use the raw output of the fully connected layer as the final output
        out = self.fc2(fcn_input)

        return out

## Select Hyperparameters

In [9]:
# Fix the random seed so that model performance is reproducible
torch.manual_seed(55)

# Initialize CNN model
model = CNNModel()

# Define learning rate, epoch and batchsize for mini-batch gradient
learning_rate = 0.001
epochs = 20
batchsize = 75

# Define loss function and optimizer
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

model

CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc2): Linear(in_features=16384, out_features=4, bias=True)
)

## Identify Tracked Values

In [10]:
train_loss_list = []
validation_accuracy_list = np.zeros((epochs,))

## Train Model

In [ ]:
import tqdm

# Convert the training, validation, testing dataset (NumPy arrays) into torch tensors
train_inputs = torch.from_numpy(train_features).float()
train_targets = torch.from_numpy(train_targets).long()

validation_inputs = torch.from_numpy(validation_features).float()
validation_targets = torch.from_numpy(validation_targets).long()

testing_inputs = torch.from_numpy(test_features).float()
testing_targets = torch.from_numpy(test_targets).long()

# split into batches
train_batches_features = torch.split(train_inputs, batchsize)
train_batches_targets = torch.split(train_targets, batchsize)

batch_split_num = len(train_batches_features)

# Training Loop ---------------------------------------------------------------------------------------

for epoch in range(epochs):
    #iteratae through the batches
    for k in range(batch_split_num):
        # typical training cycle: reset optimizer -> train -> find loss ->
        # append loss -> backwards step -> optimizer steps

        optimizer.zero_grad()

        train_batch_outputs = model(train_batches_features[k])

        loss = loss_func(train_batch_outputs, train_batches_targets[k])

        train_loss_list.append(loss.item())

        loss.backward()

        optimizer.step()

    # Compute Validation Accuracy ----------------------------------------------------------------------
    with torch.no_grad():
        # apply model
        validation_outputs = model(validation_inputs)

        # find most probable class with argmax
        correct = (torch.argmax(validation_outputs, dim=1) ==
                   validation_targets).type(torch.FloatTensor)

        print("Epoch: "+ str(epoch),
              "Validation Accuracy: " + str(np.round(correct.mean().numpy() * 100, 2))
              + '%', flush=True)

        validation_accuracy_list[epoch] = correct.mean()


Epoch: 0 Validation Accuracy: 89.0%
Epoch: 1 Validation Accuracy: 92.0%
Epoch: 2 Validation Accuracy: 91.4%
Epoch: 3 Validation Accuracy: 91.0%
Epoch: 4 Validation Accuracy: 91.6%
Epoch: 5 Validation Accuracy: 93.2%
Epoch: 6 Validation Accuracy: 93.4%


## Visualize & Evaluate Model

In [ ]:
# Seaborn for prettier plot

import seaborn as sns

sns.set(style = 'whitegrid', font_scale = 1)

In [ ]:
# Visualize training loss
plt.figure(figsize=(15, 9))
plt.subplot(2, 1, 1)
plt.plot(train_loss_list, linewidth=3, label="Training Loss")
plt.ylabel("Training Loss")
plt.xlabel("Iterations")
plt.legend()
sns.despine()

# Validation Accuracy
plt.subplot(2, 1, 2)
plt.plot(validation_accuracy_list, linewidth=3, color='gold', label="Validation Accuracy")
plt.ylabel("Validation Accuracy")
plt.xlabel("Iterations")
plt.legend()
sns.despine()

# Display the plots
plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()

In [ ]:
# Compute the testing accuracy
# Set batch size
batch_size = 200  # Adjust based on available memory

# Initialize list to store correct predictions
num_test_correct = []
y_pred_total = []

# Ensure no gradients are computed
with torch.no_grad():
    # Loop through the dataset in batches
    for i in range(0, len(testing_inputs), batch_size):
        # Select the current batch
        test_batch_inputs = testing_inputs[i:i + batch_size]
        test_batch_targets = testing_targets[i:i + batch_size]

        # Apply the model to the current batch
        y_pred_batch = model(test_batch_inputs)
        y_pred_total.append(y_pred_batch)

        # Find the correct predictions for the batch
        num_correct = (torch.argmax(y_pred_batch, dim=1) == test_batch_targets).type(torch.FloatTensor)

        num_test_correct.append(num_correct)

# Concatenate all correct predictions
total_correct = torch.cat(num_test_correct)

# Calculate the mean accuracy
testing_accuracy = total_correct.mean().item()

print("Testing Accuracy: {:.2f}%".format(testing_accuracy * 100))

In [ ]:
# accuracy of each class
all_y_pred = torch.cat(y_pred_total)
# iterate through list of classes
for i in range(4):
  # get indexes where the ith class is in the test
  testing_input_indexes = np.where(testing_targets == i)
  # find the predictions of the model on the testing inputs for those indices
  predictions = torch.argmax(all_y_pred[testing_input_indexes], dim=1).numpy()

  # get the real predictions in the same order of list
  real_targets = testing_targets[testing_input_indexes].numpy()

  # get count of where they match
  num_correct = np.sum(predictions == real_targets)
  total_per_class = len(testing_targets[testing_input_indexes])

  # calculate % correct
  print("Accuracy of", classes[i] + ":", num_correct/float(total_per_class) * 100, " %")
